In [404]:

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Loading the turnstile dataset, it has usage numbers at the different stations


In [181]:
%matplotlib inline
turn_stile_df = pd.read_csv("Turnstile_Usage_Data__2016.csv")

# Size of dataset:
print(turn_stile_df.shape)

(9492670, 11)


In [405]:
turn_stile_df.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,23:00:00,REGULAR,5572864,1881239
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,19:00:00,REGULAR,5572521,1881206
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,15:00:00,REGULAR,5571587,1881113
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,11:00:00,REGULAR,5571313,1881031
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/04/2016,08:10:05,REGULAR,5571173,1880736


Loading Information that connects the stations to a latitude longitude location

In [43]:
booth_id_df = pd.read_csv("Remote-Booth-Station - turnstiles 2013.csv")
booth_location_df = pd.read_csv("Remote-Booth-Station - stops_mgenc.csv")


In [406]:
booth_id_df.head()


,Remote,Booth,stop_id.2013,Station.2013,Line.2013,Division.2013
0,R051,A002,R11,LEXINGTON AVE,456NQR,BMT
1,R050,A004,R11,LEXINGTON AVE,456NQR,BMT
2,R079,A006,R13,5 AVE-59 ST,NQR,BMT
3,R079,A007,R13,5 AVE-59 ST,NQR,BMT
4,R080,A010,R14,57 ST-7 AVE,NQR,BMT


In [407]:
booth_location_df.head()

,stop_id,stop_name,stop_lat,stop_lon,stop_cd,stop_ct,stop_br
0,101,242ND STREET-BWAY (Van Cortland Park),40.889248,-73.898583,na,na,Bronx
1,103,238TH STREET-BWAY,40.884653,-73.900878,na,na,Bronx
2,104,231ST STREET-BWAY,40.878856,-73.904834,na,na,Bronx
3,106,225TH STREET-BWAY (Marble Hill),40.874561,-73.909831,na,na,Bronx
4,107,215TH STREET-10 AV,40.869444,-73.915279,NaN,NaN,NaN


Combining the two tables by the ID of the stop

In [408]:
combined_location_df = pd.merge(booth_id_df, booth_location_df, left_on='stop_id.2013', right_on='stop_id', how='left')


Removing Columns that are not needed

In [413]:
combined_location_df = combined_location_df[['Remote', 'Booth', 'Station.2013', 'stop_name', 'stop_lat', 'stop_lon']]

In [414]:
combined_location_df[combined_location_df.Remote == 'R051']

,Remote,Booth,Station.2013,stop_name,stop_lat,stop_lon
0,R051,A002,LEXINGTON AVE,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
577,R051,R245,59 ST,59TH STREET - LEXINGTON,40.762526,-73.967967
578,R051,R245A,59 ST,59TH STREET - LEXINGTON,40.762526,-73.967967


In [415]:
combined_location_df = combined_location_df.groupby('Remote').apply(lambda x: x.iloc[0])

In [416]:
combined_location_df.head()

,Remote,Booth,Station.2013,stop_name,stop_lat,stop_lon
Remote,,,,,,
R001,R001,A058,WHITEHALL ST,WHITEHALL ST - SOUTH FERRY,40.703087,-74.012994
R002,R002,A077,FULTON ST,FULTON STREET,40.710374,-74.007582
R003,R003,J025,CYPRESS HILLS,CYPRESS HILLS,40.689941,-73.87255
R004,R004,J028,ELDERTS LANE,ELDERTS LANE - 75 ST,40.691324,-73.867139
R005,R005,J030,FOREST PARKWAY,FOREST PARKWAY - 85 ST,40.692435,-73.86001


Finally, connecting the turnstile data with the location data

In [417]:
turn_location_df = pd.merge(turn_stile_df, combined_location_df, left_on='Unit', right_on='Remote', how='left')


Removing useless columns

In [419]:
turn_location_df = turn_location_df[['C/A', 'Unit','SCP','Division','Description','Station', 'Line Name', 'Date', 'Time', 'Entries', 'stop_name', 'stop_lat', 'stop_lon']]

In [420]:
turn_location_df = turn_location_df.sort_values(['C/A', 'Unit','SCP', 'Date', 'Time'])
turn_location_df.head(10)


,C/A,Unit,SCP,Division,Description,Station,Line Name,Date,Time,Entries,stop_name,stop_lat,stop_lon
388,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/02/2016,03:00:00,5478180,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
387,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/02/2016,07:00:00,5478183,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
386,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/02/2016,11:00:00,5478262,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
385,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/02/2016,15:00:00,5478519,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
384,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/02/2016,19:00:00,5478940,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
383,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/02/2016,23:00:00,5479213,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
382,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/03/2016,03:00:00,5479279,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
381,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/03/2016,07:00:00,5479282,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
380,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/03/2016,11:00:00,5479344,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
379,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,01/03/2016,15:00:00,5479558,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258


Only considering entries for the hour 23:00, this is a hack!


In [421]:
turn_23 = turn_location_df[turn_location_df.Time == '23:00:00']

Creating a Station location dataframe

In [437]:
station_location = turn_23[['Station', 'stop_lat', 'stop_lon']]

In [438]:
station_location = station_location.groupby('Station').apply(lambda x:x.iloc[0])


In [439]:
station_location.head()

,Station,stop_lat,stop_lon
Station,,,
1 AV,1 AV,40.730953,-73.981628
103 ST-CORONA,103 ST-CORONA,40.749865,-73.8627
104 ST,104 ST,40.695178,-73.84433
110 ST,110 ST,40.79502,-73.94425
111 ST,111 ST,40.697418,-73.836345


In [440]:
import datetime

In [441]:
turn_23['Date'] = pd.to_datetime(turn_23.Date)

/Users/leifur/smartcities_modelteam03/venv/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [442]:
turn_23.head()

,C/A,Unit,SCP,Division,Description,Station,Line Name,Date,Time,Entries,stop_name,stop_lat,stop_lon
383,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-02,23:00:00,5479213,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
377,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-03,23:00:00,5480016,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
371,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-04,23:00:00,5481770,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
365,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-05,23:00:00,5483391,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258
359,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-06,23:00:00,5485169,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258


Finding the diffs!

In [448]:
turn_23['day_before'] = turn_23.Date.map(lambda x: x - datetime.timedelta(days=1))

/Users/leifur/smartcities_modelteam03/venv/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [449]:
turn_23.head()

,C/A,Unit,SCP,Division,Description,Station,Line Name,Date,Time,Entries,stop_name,stop_lat,stop_lon,day_before
383,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-02,23:00:00,5479213,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-01
377,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-03,23:00:00,5480016,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-02
371,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-04,23:00:00,5481770,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-03
365,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-05,23:00:00,5483391,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-04
359,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-06,23:00:00,5485169,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-05


Creating a copy of the dataframe to join on

In [450]:
day_before = turn_23[['C/A','Unit', 'SCP','Division','Description', 'Date', 'Entries']]
day_before = day_before.rename(columns= {'Entries': 'before_entries', 'Date':'day_before'})


In [451]:
day_before.head()

,C/A,Unit,SCP,Division,Description,day_before,before_entries
383,A002,R051,02-00-00,BMT,REGULAR,2016-01-02,5479213
377,A002,R051,02-00-00,BMT,REGULAR,2016-01-03,5480016
371,A002,R051,02-00-00,BMT,REGULAR,2016-01-04,5481770
365,A002,R051,02-00-00,BMT,REGULAR,2016-01-05,5483391
359,A002,R051,02-00-00,BMT,REGULAR,2016-01-06,5485169


Joining the usage on day and day before

In [452]:
comb = pd.merge(turn_23, day_before, on = ['C/A','Unit', 'SCP','Division','Description', 'day_before'], how='left')

comb = comb.sort_values(['C/A', 'Unit', 'SCP','Date'])




In [453]:
comb.shape

(340550, 15)

In [454]:
turn_23.shape

(309797, 14)

In [455]:
comb.head()

,C/A,Unit,SCP,Division,Description,Station,Line Name,Date,Time,Entries,stop_name,stop_lat,stop_lon,day_before,before_entries
0,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-02,23:00:00,5479213,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-01,NaN
1,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-03,23:00:00,5480016,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-02,5479213.0
2,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-04,23:00:00,5481770,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-03,5480016.0
3,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-05,23:00:00,5483391,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-04,5481770.0
4,A002,R051,02-00-00,BMT,REGULAR,59 ST,NQR456,2016-01-06,23:00:00,5485169,LEXINGTON AVENUE - 59 ST,40.76266,-73.967258,2016-01-05,5483391.0


The diff of today cumulative entries minus yesterday

In [456]:
comb['entries_diff'] = comb.Entries - comb.before_entries

Does it make sense?:

In [458]:
print comb.entries_diff.max()
print comb.entries_diff.min()

1675034616.0
-1877466424.0


They don't! Let's throw away the entries that don't make sense! (hacky hack)

In [459]:
comb_sub = comb[comb.entries_diff<10000]

In [460]:
comb_sub = comb_sub[0<comb_sub.entries_diff]

Sum across turnstiles at stations for a particular dates

In [462]:
station_group = comb_sub.groupby(['Station', 'Date']).agg({'entries_diff':sum})

In [465]:
station_group['Station'] = station_group.index.map(lambda x: x[0])


In [466]:
station_group.head()

entries_diff Station
Station Date                            
1 AV    2016-01-03       14437.0    1 AV
        2016-01-04       21800.0    1 AV
        2016-01-05       22641.0    1 AV
        2016-01-06       23787.0    1 AV
        2016-01-07       23839.0    1 AV

Find the average usage for stations over days

In [467]:
final = station_group.groupby(['Station']).agg({'entries_diff':'mean'})

In [468]:
final =final.reset_index()

Get the locations back!

In [470]:
final_location = pd.merge(final, station_location, on='Station')

In [471]:
final_location = final_location.sort_values('entries_diff',ascending=False)

In [473]:
final_location.head(300)

,Station,entries_diff,stop_lat,stop_lon
39,34 ST-HERALD SQ,120875.913793,40.749567,-73.98795
41,34 ST-PENN STA,115917.241379,40.752287,-73.993391
198,TIMES SQ-42 ST,96230.508621,40.75529,-73.987495
74,86 ST,88998.215517,40.622687,-74.028398
59,59 ST,79695.431034,40.76266,-73.967258
31,23 ST,76994.137931,40.741303,-73.989344
60,59 ST COLUMBUS,73811.508621,40.768247,-73.981929
143,GRD CNTRL-42 ST,68982.008621,40.751776,-73.976848
115,CHAMBERS ST,68055.068966,40.713243,-74.003401
134,FLUSHING-MAIN,61454.405172,40.7596,-73.83003


In [398]:
final_location.to_csv('station_traffic_location.csv')